### Data Anotation

In [ ]:
import os
file_list=os.listdir("../Done")
file_list

In [ ]:
json_audio_files = list()
for file in file_list:
    local_files = os.listdir(os.path.join("../Done", file))
    for json_file in local_files:
        if json_file.endswith(".json"):
            print(json_file)
            json_audio_files.append(os.path.join("../Done", file, json_file))

In [ ]:
import json
def get_annotated_duration(value):
    value = result.get('value')
    duration = float(value.get('end')) - float(value.get('start'))
    duration_in_hours = duration/3600
    return duration_in_hours

In [ ]:
total_annotation_duration = 0
for file in json_audio_files:
    with open(file, encoding="UTF8") as json_file:
        data = json.load(json_file)
        completions = data[0].get('completions')
        annotations = data[0].get('annotations')
        if completions:
            audio_results = completions[0]['result']
            for result in audio_results:
                if result.get('from_name') == 'transcription':
                    duration_in_hours = get_annotated_duration('value')
                    total_annotation_duration += duration_in_hours
        elif annotations:
            audio_results = annotations[0]['result']
            for result in audio_results:
                if result.get('from_name') == 'transcription':
                    duration_in_hours = get_annotated_duration('value')
                    total_annotation_duration += duration_in_hours
print(f"Total Anotated Speech: {total_annotation_duration}")

In [ ]:
man_readers_speech = 0
woman_readers_speech = 0

for file in json_audio_files:
    with open(file, encoding="UTF8") as json_file:
        data = json.load(json_file)
        completions = data[0].get('completions')
        annotations = data[0].get('annotations')
        if completions:
            audio_results = completions[0]['result']
            for result in audio_results:
                if result.get('from_name') == 'labels':
                    value = result.get('value')
                    reader = value.get('labels')[0]
                    duration = float(value.get('end')) - float(value.get('start'))
                    duration_in_hours = duration/3600
                    if reader.startswith("Կին"):
                        woman_readers_speech += duration_in_hours
                    elif reader.startswith("Տղամարդ"):
                        man_readers_speech += duration_in_hours
                    
        elif annotations:
            audio_results = annotations[0]['result']
            for result in audio_results:
                if result.get('from_name') == 'labels':
                    value = result.get('value')
                    reader = value.get('labels')[0]
                    duration = float(value.get('end')) - float(value.get('start'))
                    duration_in_hours = duration/3600
                    if reader.startswith("Կին"):
                        woman_readers_speech += duration_in_hours
                    elif reader.startswith("Տղամարդ"):
                        man_readers_speech += duration_in_hours
                        
print(f"Total Anotated Man's Speech: {man_readers_speech}" , f"\nTotal Anotated Woman's Speech: {woman_readers_speech}")

In [ ]:
summary_data = dict()
for file in json_audio_files:
    with open(file, encoding="UTF8") as json_file:
        data = json.load(json_file)
        completions = data[0].get('completions')
        annotations = data[0].get('annotations')
        if completions:
            reader_speech_time = 0 
            audio_results = completions[0]['result']
            for result in audio_results:
                if result.get('from_name') == 'labels':
                    value = result.get('value')
                    reader = value.get('labels')[0]
                    duration = float(value.get('end')) - float(value.get('start'))
                    duration_in_hours = duration/3600
                    reader_speech_time += duration_in_hours 
        elif annotations:
            reader_speech_time = 0
            audio_results = annotations[0]['result']
            for result in audio_results:
                if result.get('from_name') == 'labels':
                    value = result.get('value')
                    reader = value.get('labels')[0]
                    duration = float(value.get('end')) - float(value.get('start'))
                    duration_in_hours = duration/3600
                    reader_speech_time += duration_in_hours
        
        summary_data[reader] = [round(reader_speech_time, 2)] if reader not in summary_data.keys() else [round((summary_data[reader][0] + reader_speech_time), 2)]
print(summary_data)

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(summary_data, orient='index', columns=['Speech_Duration']).reset_index()
df.rename(columns={'index': 'Reader'}, inplace=True)

In [ ]:
sorted_df = df.sort_values(by=['Reader', 'Speech_Duration']).reset_index(drop=True)

In [ ]:
sorted_df